In [200]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import pandas as pd
import requests
import math

In [75]:
reports_r = requests.get('https://peridash.ml/api/reports')
measurements_r = requests.get('https://peridash.ml/api/measurements')
epsas_r = requests.get('https://peridash.ml/api/epsas')
variables_r = requests.get('https://peridash.ml/api/variables')
indicators_r = requests.get('https://peridash.ml/api/indicators')

In [76]:
reports_df = pd.read_json(reports_r.text)
measurements_df = pd.read_json(measurements_r.text)
epsas_df = pd.read_json(epsas_r.text)
variables_df = pd.read_json(variables_r.text)
indicators_df = pd.read_json(indicators_r.text)

complete_reports_df = pd.merge(reports_df, epsas_df, left_on='epsa', right_on='url')
complete_measurements_df = pd.merge(measurements_df, epsas_df, left_on='epsa', right_on='url')

In [3]:
init_notebook_mode(connected=True)

In [77]:
ind_names = [
    'Rendimiento actual de la fuente', 'Uso eficiente del recurso',
    'Cobertura de muestras de agua potable',
    'Conformidad de los análisis de agua potable realizados',
    'Dotación', 'Continuidad por racionamiento', 'Continuidad por corte',
    'Cobertura del servicio de agua potable',
    'Cobertura del servicio de alcantarillado sanitario',
    'Cobertura de micromedición',
    'Incidencia extracción de agua cruda subterránea ',
    'Índice de tratamiento de agua residual', 'Control de agua residual',
    'Capacidad instalada de planta de tratamiento de agua potable',
    'Capacidad instalada de planta de tratamiento de agua residual ',
    'Presión del servicio de agua potable',
    'Índice de agua no contabilizada en producción',
    'Índice de agua no contabilizada en la red',
    'Densidad de fallas en tuberías de agua potable',
    'Densidad de fallas en conexiones de agua potable',
    'Densidad de fallas en tuberías de agua residual',
    'Densidad de fallas en conexiones de agua residual',
    'Índice de operación eficiente', 'Prueba ácida',
    'Eficiencia de recaudación', 'Índice de endeudamiento total', 'Tarifa media',
    'Costo unitario de operación', 'Índice de ejecución de inversiones',
    'Personal calificado', 'Número de empleados por cada 1000 conexiones',
    'Atención de reclamos'
]
ind_units = ['%', '%', '%', '%', 'l/hab/día', 'hr/día', '%', '%', '%', '%',
             '%', '%', '%', '%', '%', '%', '%', '%', 'fallas/100km',
             'fallas/1000conex.', 'fallas/100km', 'fallas/1000conex.', '%', '-',
             '%', '%', '%CUO(Bs.)', '%TM(Bs.)', '%', '%', 'empleados/1000conex.', '%']

colors = [
    '#1f77b4','#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

def get_ind_name(ind):
    return ind_names[int(ind[3:])-1]

def get_ind_unit(ind):
    return ind_units[int(ind[3:])-1]

In [205]:
num_indicators = 32
cmdf = complete_measurements_df
ind_list = [f'ind{i+1}' for i in range(num_indicators)]

def get_data_rating(epsa_code, year):
    values = list(cmdf[(cmdf.year == year) & (cmdf.code == epsa_code)][ind_list].iloc[0])
    num_reported = sum([0 if math.isnan(val) else 1 for val in values])
    return num_reported / num_indicators * 100

In [254]:
selected_category = 'B'
cmdf = complete_measurements_df
filtered_df = cmdf[cmdf.category == selected_category]
epsa_codes = list(filtered_df.code.unique())
years = list(filtered_df.sort_values('year').year.unique())

scat1 = go.Scatter(
    x= [2014] * len(epsa_codes),
    y= list(filtered_df[filtered_df.year == 2014].ind1),
    text= epsa_codes,
    mode='markers+text',
    showlegend=False,
)

data = [scat1]

for epsa_code in epsa_codes:
    line_trace = go.Scatter(
        x= years,
        y= [get_data_rating(epsa_code, year) for year in years],
#         y= list(filtered_df[filtered_df.code == epsa_code].sort_values('year').ind1),
        mode='lines',
        name=epsa_code,
#         hoverinfo='none',
    )
    data.append(line_trace)

In [251]:
layout = dict(
    xaxis= dict(
        title='Años',
        range=[2013.8, 2017.2],
        autorange= False,
        dtick=1,
    ),
    yaxis= dict(
        title='Porcentaje de variables reportadas (%)',
#         dtick=1,
#         range=[50,100],
#         autorange=False,
    ),
    title= f'Ranking EPSAS - Porcentaje de variables reportadas<br>Categoría {selected_category}',
    updatemenus= [dict(
        x=0,
        y=0,
        type= 'buttons',
        buttons=[dict(
            label='Animar',
            method='animate',
            args= [None],
        )],
    )],
    hovermode='closest',
)

In [252]:
# frames = [
#     {'data': [go.Scatter(x=[1, 2], y=[1, 2], text=['A', 'B'])]},
#     {'data': [go.Scatter(x=[1, 4], y=[1, 4], text=['A', 'B'])]},
#     {'data': [go.Scatter(x=[3, 4], y=[3, 4], text=['A', 'B'])]},
# ]


colors = [
    '#1f77b4', '#ff7f0e','#2ca02c',
    '#d62728', '#9467bd', '#8c564b',
    '#e377c2', '#7f7f7f', '#bcbd22','#17becf'
]

colors = colors * 3

frames = [{'data': [go.Scatter(
    x=[year]*len(epsa_codes),
    y=[get_data_rating(epsa_code, year) for epsa_code in epsa_codes],
#     y=list(filtered_df[filtered_df.year == year].ind1),
    text=epsa_codes,
    textposition='top center',
    mode='markers',
    marker=dict(symbol='circle', size=20, color=colors[1:]),
)]} for year in years]

In [255]:
fig = go.Figure(data=data, layout=layout, frames=frames)
iplot(fig)

In [256]:
import plotly.plotly as py

py.iplot(fig, filename='ranking_datos')